In [1]:
import torch
torch.cuda.set_device(2)
print(torch.cuda.current_device())

2


In [2]:
%run prepare_data.py
%run architectures.py

### Mura

#### MobileNet

In [3]:
PATH = Path('/home/rimmanni/data/mura')
valid_path = PATH/"valid_250_200"
valid_dataset = datasets.ImageFolder(valid_path, transforms.Compose([
    transforms.ToTensor()
]))
sampled = torch.utils.data.RandomSampler(valid_dataset, replacement=True)

In [4]:
p = Path("Scaling_experiments/Mura/")
ls = list(p.iterdir())
models = [str(s) for s in ls if '_' in str(s) and 'ipynb' not in str(s) and '.csv' not in str(s) 
          and 'Mobile' in str(s)]

In [5]:
len(models)

24

In [6]:
data = []

for model_p in models:
    w = float(model_p.split('_')[-2])
    d = float(model_p.split('_')[-1])
    model = MobileNet(num_classes=1, width_mult=w, depth_mult=d).cuda()
    load_model(model, model_p)
    for i in range(15):
        ids = list(iter(sampled))
        valid_ds_sampled = torch.utils.data.Subset(valid_dataset, ids)
        valid_ds_sampled.samples = [valid_dataset.samples[i] for i in ids]
        valid_loader = DataLoader(valid_ds_sampled, batch_size=32,num_workers=4)
        _, score = val_metrics_mura(model, valid_loader, valid_ds_sampled)
        data.append([model_p, score])
#         print(model_p, val_metrics_mura(model, valid_loader, valid_ds_sampled))

In [7]:
mura_mobile = pd.DataFrame(data=data, columns=['model', 'score'])

In [8]:
mura_mobile.to_csv('bootstrap/mura_mobile.csv', index=False)

#### Resntet_Depth

In [9]:
p = Path("Scaling_experiments/Mura/")
ls = list(p.iterdir())
models = [str(s) for s in ls if 'ResDep' in str(s)]
len(models)

24

In [10]:
depths = [[[[64, 2], [128, 2]], [[256, 2], [512, 1]]],
          [[[64, 2], [128, 2]], [[256, 1], [512, 1]]],
          [[[64, 2], [128, 1]], [[256, 1], [512, 1]]],
          [[[64, 2], [128, 1]], [[256, 2], [512, 1]]],
          [[[64, 1], [128, 1]], [[256, 2], [512, 1]]],
          [[[64, 1], [128, 1]], [[256, 1], [512, 1]]],
         ]

In [11]:
data = []

for model_p in models:
    w = float(model_p.split('_')[-2])
    i = int(model_p.split('_')[-1])
    d = depths[i]
    model = resnet18(block=depthwise_block, width_mult=w, 
                     inverted_residual_setting1=d[0], 
                     inverted_residual_setting2=d[1]).cuda()
    load_model(model, model_p)
    for i in range(15):
        ids = list(iter(sampled))
        valid_ds_sampled = torch.utils.data.Subset(valid_dataset, ids)
        valid_ds_sampled.samples = [valid_dataset.samples[i] for i in ids]
        valid_loader = DataLoader(valid_ds_sampled, batch_size=32,num_workers=4)
        _, score = val_metrics_mura(model, valid_loader, valid_ds_sampled)
        data.append([model_p, score])
#         print(model_p, val_metrics_mura(model, valid_loader, valid_ds_sampled))

In [12]:
mura_res_depth = pd.DataFrame(data=data, columns=['model', 'score'])

In [13]:
mura_res_depth.to_csv('bootstrap/mura_res_depth.csv', index=False)